In [48]:
import cv2
from skimage.io import imread, imsave
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import numpy as np
from PIL import Image
import tqdm
import pandas as pd
import pytorch_lightning as pl
import os
import copy
from sklearn.utils.class_weight import compute_class_weight

In [14]:
! git clone https://github.com/jlevy44/PreliminaryGenerativeHistoPath


Cloning into 'PreliminaryGenerativeHistoPath'...
remote: Enumerating objects: 310, done.
remote: Counting objects: 100% (310/310), done.
remote: Compressing objects: 100% (309/309), done.
remote: Total 310 (delta 1), reused 306 (delta 0), pack-reused 0
Receiving objects: 100% (310/310), 15.28 MiB | 11.58 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [16]:
def load_convert_img(filepath=""):
    img=cv2.imread(filepath)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return img

def make_image_mask(im):
    image=im[:,:int(im.shape[1]/2)]
    mask=im[:,int(im.shape[1]/2):]
    final_mask=np.zeros(image.shape[:2]).astype(int)
    final_mask[(mask[...,1]>220)&(mask[...,2]<50)]=1
    final_mask[(mask[...,2]>220)&(mask[...,1]<50)]=2
    return image,final_mask

def load_return_data(filepath=''):
    im=load_convert_img(filepath)
    image,final_mask=make_image_mask(im)
    return image,final_mask

In [43]:
img_masks_train=[load_return_data(f) for f in sorted(glob.glob("PreliminaryGenerativeHistoPath/cyto2label_public/train/*.png"))]
img_masks_val=[load_return_data(f) for f in sorted(glob.glob("PreliminaryGenerativeHistoPath/cyto2label_public/val/*.png"))]

train_imgs=list(map(lambda x: cv2.resize(x[0],(256,256),interpolation=cv2.INTER_CUBIC),img_masks_train))
train_masks=list(map(lambda x: np.concatenate([cv2.resize(x[1].astype(np.uint8),(256,256),interpolation=cv2.INTER_NEAREST).astype(np.uint8)[...,np.newaxis],np.zeros((256,256,2),dtype=np.uint8)],axis=2),img_masks_train))

val_imgs=list(map(lambda x: cv2.resize(x[0],(256,256),interpolation=cv2.INTER_CUBIC),img_masks_val))
val_masks=list(map(lambda x: np.concatenate([cv2.resize(x[1].astype(np.uint8),(256,256),interpolation=cv2.INTER_NEAREST).astype(np.uint8)[...,np.newaxis],np.zeros((256,256,2),dtype=np.uint8)],axis=2),img_masks_val))

In [44]:
! mkdir -p train/imgs train/labels val/imgs val/labels 

In [47]:
for i,(im,lbl) in enumerate(zip(train_imgs,train_masks)):
    imsave(f"train/imgs/{i}.png",im)
    imsave(f"train/labels/{i}.png",lbl)
    
for i,(im,lbl) in enumerate(zip(val_imgs,val_masks)):
    imsave(f"test/imgs/{i}.png",im)
    imsave(f"test/labels/{i}.png",lbl)

/Users/joshualevy/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: train/labels/0.png is a low contrast image
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/joshualevy/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: train/labels/1.png is a low contrast image
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/joshualevy/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: train/labels/2.png is a low contrast image
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/joshualevy/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: train/labels/3.png is a low contrast image
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/joshualevy/opt/anaconda3/envs/py37/lib/python3.7/site-package